In [77]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit,KFold
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from utils import score, create_submission

def train(df: pd.DataFrame):
    target = 'TOTAL_FIRE_SIZE'
    features = features = [col for col in df.columns if col != target]

    # Sort by time to ensure valid time-series splits
    X = df[features]
    y = df[target]
    
    def custom_cv_metric(estimator, X_cv, y_cv):
        """
        Implements the metric:
            mean( min( |log(y_pred / y_true)|, 10 ) )
        and returns its negative for hyperparameter tuning.
        """        
        y_pred = estimator.predict(X_cv)
        eps = 1e-15
        y_pred = np.maximum(y_pred, eps)
        y_true = np.maximum(y_cv, eps)
        
        log_errors = np.abs(np.log(y_pred / y_true))
        log_errors_clamped = np.minimum(log_errors, 10.0)
        score_value = np.mean(log_errors_clamped)
        return -score_value
    
    def squared_log_error_obj(preds, dtrain):
        """
        Custom objective function for squared error in log-space:
        L = (log(pred) - log(label))^2
        """
        print(dtrain)
        print(len(dtrain))
        labels = dtrain.get_label()
        eps = 1e-15

        preds = np.maximum(preds, eps)
        diff = np.log(preds) - np.log(labels)
        grad = 2.0 * diff / preds
        hess = 2.0 * (1.0 - diff) / (preds ** 2)
        return grad, hess

    
    # Define the parameter search space using skopt spaces.
    param_space = {
        'max_depth': Integer(3, 15),
        'min_child_weight': Integer(1, 10),
        'gamma': Real(0, 5, prior='uniform'),
        'learning_rate': Real(0.01, 0.2, prior='log-uniform'),
        'subsample': Real(0.7, 1.0),
        'colsample_bytree': Real(0.7, 1.0),
        'colsample_bylevel': Real(0.7, 1.0),
        'reg_alpha': Real(0, 1, prior='uniform'),
        'reg_lambda': Real(0, 1, prior='uniform'),
        'max_delta_step': Integer(0, 10),
        'n_estimators': Integer(50, 300)
    }
    
    # Use TimeSeriesSplit to respect time order and avoid leakage.
    cv = TimeSeriesSplit(n_splits=5)
    
    # Initialize the XGBoost regressor (scikit-learn API).
    model = XGBRegressor(
        objective='reg:squaredlogerror',
        eval_metric='rmse',
        random_state=42
    )
    
    # Set up Bayesian optimization with BayesSearchCV.
    bayes_search = BayesSearchCV(
        estimator=model,
        search_spaces=param_space,
        n_iter=100,
        scoring=custom_cv_metric,
        cv=cv,
        verbose=1,
        random_state=42,
        n_jobs=-1
    )
    
    # Run the Bayesian search.
    bayes_search.fit(X, y)
    
    print("Best parameters found:", bayes_search.best_params_)
    # Our scorer returns negative custom score, so we multiply by -1.
    print("Best CV custom score:", -bayes_search.best_score_)
    
    # Retrieve the best estimator.
    best_model = bayes_search.best_estimator_
    return best_model


In [ ]:
data_path = 'datasets/dataset_final_3_target_encoding.csv'
df = pd.read_csv(data_path)
best_model = train(df)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [ ]:
def calc_log_clamped_score(y_true, y_pred):
    """
    Implements: mean( min( |log(y_pred / y_true)|, 10 ) )
    """
    eps = 1e-15
    y_pred = np.maximum(y_pred, eps)
    y_true = np.maximum(y_true, eps)
    
    log_errors = np.abs(np.log(y_pred / y_true))
    log_errors_clamped = np.minimum(log_errors, 10.0)
    return np.mean(log_errors_clamped)

def evaluate_train_test_scores(df, best_model):
    """
    Perform manual time-series CV with the best-found model to compute
    separate train/test scores for each fold.
    """
    target = 'TOTAL_FIRE_SIZE'
    features = [col for col in df.columns if col != target]

    # Sort for time-based CV
    X = df[features]
    y = df[target]
    
    tscv = TimeSeriesSplit(n_splits=10)
    
    train_scores, test_scores = [], []
    for train_idx, test_idx in tscv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Re-initialize a new model with the same parameters
        # (Alternatively, you can use sklearn.base.clone(best_model))
        model_fold = XGBRegressor(**best_model.get_params())
        model_fold.fit(X_train, y_train)
        
        # Predictions
        y_train_pred = model_fold.predict(X_train)
        y_test_pred = model_fold.predict(X_test)
        
        # Calculate metrics
        fold_train_score = calc_log_clamped_score(y_train, y_train_pred)
        fold_test_score  = calc_log_clamped_score(y_test, y_test_pred)
        
        train_scores.append(fold_train_score)
        test_scores.append(fold_test_score)
    
    # Print or return the results
    print("Train scores (each fold):", train_scores)
    print("Test scores (each fold): ", test_scores)
    print("Mean train score:", np.mean(train_scores))
    print("Mean test score: ", np.mean(test_scores))
    
    return train_scores, test_scores


In [ ]:
train_scores, test_scores = evaluate_train_test_scores(df, best_model)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[0.5 0.5 0.5 ... 0.5 0.5 0.5]
<class 'numpy.ndarray'>
[0.5048755  0.5048755  0.50098765 ... 0.49159354 0.50098765 0.5048755 ]
<class 'numpy.ndarray'>
[0.50956786 0.5101983  0.50176215 ... 0.4946046  0.50176215 0.50956786]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.nda

/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/Users/mflodzinski/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2

[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan][nan nan nan ... nan nan nan]

<class 'numpy.ndarray'><class 'numpy.ndarray'>

[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan

/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/Users/mflodzinski/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan]
  warnings.warn(
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/Users/mfl

[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan

...

[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan

/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/Users/mflodzinski/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan]
  warnings.warn(
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encounter

[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>[nan nan nan ... nan nan nan]

<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan]
<class 'numpy.ndarray'>
[nan nan nan ... nan nan nan

/Users/mflodzinski/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan]
  warnings.warn(
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/var/folders/2p/xyrcbmm94rx7jntjnk1rd9980000gn/T/ipykernel_52234/2438856998.py:50: RuntimeWarning: invalid value encountered in log
/Users/mflodzinski/anaconda3/envs/ml_env/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1103: UserWar

ValueError: Input y contains NaN.

In [ ]:
def retrain_and_submit(df: pd.DataFrame, best_model: xgb.XGBRegressor):
    """
    Retrains the best_model on the full dataset and creates a submission file.
    """

    target = 'total_fire_size'
    features = [col for col in df.columns if col != target]

    df = df.sort_values("month")
    X_full = df[features]
    y_full = df[target]

    best_model.fit(X_full, y_full)
    y_pred_full = best_model.predict(X_full)

    submission_df = df[['STATE', 'month']].copy()
    submission_df['total_fire_size'] = y_pred_full

    create_submission(submission_df)

if __name__ == '__main__':
    submisstion_dataset = ...
    df = pd.read_csv(submisstion_dataset)
    best_model = train(df)
    retrain_and_submit(df, best_model)


ValueError: Invalid file path or buffer object type: <class 'ellipsis'>